In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm

import variables as var
import qlearning as ql
import state as st
import io as io

In [2]:
T_Y = var.T_Y
T_G = var.T_G
T_R = var.T_R
total_T = T_Y + T_G + T_R

d_lower = var.d_lower
d_upper = var.d_upper
v_lower = var.v_lower
v_upper = var.v_upper

delta_t = var.delta_t

phi_to_T_dict = {'Y': T_Y, 'G': T_G, 'R': T_R}
phi_to_next_phi_dict = {'G':'Y', 'Y':'R', 'R':'G'}
phi_prev_phi_dict = var.phi_prev_phi_dict
#(0.4, 0.1, 0.05)
std_d= 0.4#var.std_d
std_v= 0.1#var.std_v
std_t_phi= 0.05#var.std_t_phi

trials = 50000000#var.trials
episodes = var.episodes

learning_rate = 0.001#var.learning_rate
discount_factor = 0.95#var.discount_factor
epsilon = var.epsilon

actions = var.actions
discrete_d = var.discrete_d
v_max= var.v_max
discrete_v = var.discrete_v
discrete_phi_t = var.discrete_phi_t
# Q_table = var.Q_table


In [3]:
Q_table = np.zeros((len(discrete_d)*len(discrete_v)*len(discrete_phi_t), len(actions)))
np.random.seed()
accum_rewards = []
prev_action = 0
for trial in tqdm(range(trials)):
#     state = func.random_init_state(d_lower, d_upper, v_lower, v_upper, T_Y, T_G, T_R)
#     idx_i = func.map_state_to_idx(state, discrete_d, discrete_v, discrete_phi_t, T_Y, T_R)
    idx_i = np.random.randint(0, Q_table.shape[0], 1)
    state = st.map_idx_to_state(idx_i, discrete_d, discrete_v, discrete_phi_t)
    distance = state[0]
    velocity = state[1]
    accum_r = 0
    for episode in range(episodes):
        
        if distance<=0:
            break
        
        action, action_idx = ql.action_selection(Q_table[idx_i], velocity, actions, epsilon, v_max,1)
        q = Q_table[idx_i, action_idx]
        
        state_new = st.update_state(state, action, delta_t, phi_to_T_dict, phi_to_next_phi_dict, std_d, std_v, std_t_phi,1)
        idx_i_new = st.map_state_to_idx(state_new, discrete_d, discrete_v, discrete_phi_t, T_Y, T_R, phi_prev_phi_dict)
#         mapped_state= func.map_idx_to_state(idx_i_new, discrete_d, discrete_v, discrete_phi_t)
        distance = state_new[0]
        velocity = state_new[1]
        phi = state_new[2]
        t_phi = state_new[3]
        
        reward = ql.reward_function_smooth(distance, velocity, phi, t_phi, prev_action, action, v_max, T_Y)
        prev_action = action
        
        td = ql.TD_function(reward, discount_factor, Q_table[idx_i_new], q)
        
        q = q + learning_rate*td
        Q_table[idx_i,action_idx] = q
        
        idx_i = idx_i_new
        state = state_new
        
        if trial%10000==0:
#             print(reward)
            accum_r+=reward
    
    if accum_r!=0:
        accum_rewards.append(accum_r)
        
io.write_list(accum_rewards, 'QLearning_accum_reward')
with open('Q_table.npy','wb') as f:
    np.save(f, Q_table)

  0%|          | 6322/50000000 [00:08<19:05:43, 727.25it/s]


KeyboardInterrupt: 